## Load Data

In [2]:
import pandas as pd

# path to data file
df_1 = pd.read_csv("/Users/murphy.chen/Desktop/data1030final_project/data/raw/Ad_table.csv", low_memory=False)
df_2 = pd.read_csv("/Users/murphy.chen/Desktop/data1030final_project/data/raw/Ad_table (extra).csv", low_memory=False)

df = pd.concat([df_1, df_2], ignore_index=True)

# check the combined data
print(df.shape)
print(list(df.columns))

(536510, 26)
['Maker', 'Genmodel', 'Genmodel_ID', 'Adv_ID', 'Adv_year', 'Adv_month', 'Color', 'Reg_year', 'Bodytype', 'Runned_Miles', 'Engin_size', 'Gearbox', 'Fuel_type', 'Price', 'Seat_num', 'Door_num', ' Genmodel', ' Genmodel_ID', 'Engine_power', 'Annual_Tax', 'Wheelbase', 'Height', 'Width', 'Length', 'Average_mpg', 'Top_speed']


## Data Cleaning

In [ ]:
# start with keeping the necessary columns
columns_to_keep = ["Maker", 
                   "Genmodel", 
                   "Color", 
                   "Reg_year", 
                   "Bodytype", 
                   "Runned_Miles", 
                   "Engin_size", 
                   "Gearbox", 
                   "Fuel_type", 
                   "Price", 
                   "Seat_num",  
                   "Door_num"]

df = df[columns_to_keep].copy()

# check the new data columns
print(df.shape)
print(list(df.columns))

(536510, 12)
['Maker', 'Genmodel', 'Color', 'Reg_year', 'Bodytype', 'Runned_Miles', 'Engin_size', 'Gearbox', 'Fuel_type', 'Price', 'Seat_num', 'Door_num']


In [7]:
# clean and standardize the names
df.columns = (
    df.columns
    .str.strip()            
    .str.lower()             
    .str.replace(" ", "_")     
    .str.replace("-", "_") 
)

# check the cleaned column names
print(df.columns.tolist())

['maker', 'genmodel', 'color', 'reg_year', 'bodytype', 'runned_miles', 'engin_size', 'gearbox', 'fuel_type', 'price', 'seat_num', 'door_num']
